In [ ]:
#|default_exp core

# Claudio

## Setup

- basic and streaming chat
- images
- tool use

In [ ]:
import os
# os.environ['ANTHROPIC_LOG'] = 'debug'

In [ ]:
#| export
import tokenize, ast, inspect, inspect, typing
import xml.etree.ElementTree as ET, json
try: from IPython.display import Markdown
except: Markdown=None

from anthropic import Anthropic
from anthropic.types import Usage
from inspect import Parameter
from io import BytesIO

from fastcore.docments import docments
from fastcore.utils import *

In [ ]:
#| export
models = 'claude-3-opus-20240229','claude-3-sonnet-20240229','claude-3-haiku-20240307'

In [ ]:
model = models[1]

## Client

In [ ]:
#| export
def mk_msg(content, role='user', **kw):
    "Helper to create a `dict` appropriate for a Claude message"
    return dict(role=role, content=content, **kw)

In [ ]:
#| export
def mk_msgs(msgs, **kw):
    "Helper to set 'assistant' role on alternate messages"
    if isinstance(msgs,str): msgs=[msgs]
    return [mk_msg(o, ('user','assistant')[i%2], **kw) if isinstance(o,str) else o
            for i,o in enumerate(msgs)]

In [ ]:
#| export
def contents(r):
    "Help to get the contents from Claude response `r`"
    return r.content[0].text.strip()

In [ ]:
cli = Anthropic()

In [ ]:
#|export
@patch
def __add__(self:Usage, b):
    return Usage(input_tokens=self.input_tokens+b.input_tokens, output_tokens=self.output_tokens+b.output_tokens)

@patch(as_prop=True)
def total(self:Usage): return self.input_tokens+self.output_tokens

@patch
def __repr__(self:Usage): return f'In: {self.input_tokens}; Out: {self.output_tokens}; Total: {self.total}'

In [ ]:
#| export
class Client:
    def __init__(self, model, cli=None):
        "Basic Anthropic messages client"
        self.model,self.use = model,Usage(input_tokens=0,output_tokens=0)
        self.c = (cli or Anthropic())

    def _r(self, r):
        self.result = r
        self.use += r.usage
        return r

    def __call__(self, msgs, sp='', temp=0, maxtok=4096, stop=None, **kw):
        r = self.c.beta.tools.messages.create(
            model=self.model, messages=mk_msgs(msgs), max_tokens=maxtok, system=sp, temperature=temp, stop_sequences=stop, **kw)
        return self._r(r)

    def stream(self, msgs, sp='', temp=0, maxtok=4096, stop=None, **kw):
        msgs = mk_msgs(msgs)
        with self.c.messages.stream(model=self.model, messages=mk_msgs(msgs), max_tokens=maxtok,
                                    system=sp, temperature=temp, stop_sequences=stop, **kw) as s:
            yield from s.text_stream
            return self._r(s.get_final_message())

In [ ]:
c = Client(models[-1])

In [ ]:
for o in c.stream('Hi'): print(o, end='')

Hello! How can I assist you today?

In [ ]:
c('Hi')

ToolsBetaMessage(id='msg_016a7tAmwNzGF7bMoySs2pDk', content=[TextBlock(text='Hello! How can I assist you today?', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=In: 8; Out: 12; Total: 20)

In [ ]:
c.use

In: 16; Out: 24; Total: 40

## Tool use

In [ ]:
#| export
def get_schema(f):
    tmap = {int:"integer", float:"number", str:"string", bool:"boolean", list:"array", dict:"object"}
    paramd = dict(type="object", properties={}, required=[])
    schema = dict(name=f.__name__, description=f.__doc__, input_schema=paramd)

    def _types(anno):
        if getattr(anno, '__origin__', None) in  (list,tuple): return "array", tmap.get(anno.__args__[0], "object")
        else: return tmap.get(anno, "object"), None

    for pname, pinfo in docments(f, full=True).items():
        if pname == "return": continue
        paramt,itemt = _types(pinfo.anno)
        pschema = dict(type=paramt, description=pinfo.docment)
        if itemt: pschema["items"] = {"type": itemt}
        if pinfo.default is not Parameter.empty: pschema["default"] = pinfo.default
        else: paramd["required"].append(pname)
        paramd["properties"][pname] = pschema

    return schema

In [ ]:
def silly_sum(
    # First thing to sum
    a:int,
    # Second thing to sum
    b:int=1,
    # A pointless argument
    c:list[int]=None,
# The sum of the inputs
) -> int:
    "Adds a + b"
    return a + b

In [ ]:
get_schema(silly_sum)

{'name': 'silly_sum',
 'description': 'Adds a + b',
 'input_schema': {'type': 'object',
  'properties': {'a': {'type': 'integer', 'description': 'First thing to sum'},
   'b': {'type': 'integer',
    'description': 'Second thing to sum',
    'default': 1},
   'c': {'type': 'array',
    'description': 'A pointless argument',
    'items': {'type': 'integer'},
    'default': None}},
  'required': ['a']}}

In [ ]:
def sums(
    # First thing to sum
    a:int,
    # Second thing to sum
    b:int=1
# The sum of the inputs
) -> int:
    "Adds a + b"
    return a + b

In [ ]:
pr = "What is 6+3?"
sp = "You must use the `sums` function instead of adding yourself, but don't mention what tools you use."
tools=[get_schema(sums)]

In [ ]:
msgs = mk_msgs(pr)

In [ ]:
r = c(msgs, sp=sp, tools=tools)

In [ ]:
def call_func(c):
    fc = c.content[0]
    f = globals()[fc.name]
    return f(**fc.input)

In [ ]:
res = call_func(r)
res

9

In [ ]:
msgs.append(mk_msg(r.content, role=r.role))

In [ ]:
msgs.append(mk_msg([dict(type="tool_result", tool_use_id=r.content[0].id, content=str(res))]))

In [ ]:
res = c(msgs, sp=sp, tools=tools)

In [ ]:
contents(res)

'The sum of 6 and 3 is 9.'

## XML helpers

In [ ]:
#| export
def hl_md(s, lang='xml'):
    "Syntax highlight `s` using `lang`"
    if Markdown: return Markdown(f'```{lang}\n{s}\n```')
    print(s)

In [ ]:
#| export
def to_xml(node, hl=False):
    "Convert `node` to an XML string"
    def mk_el(tag, cs, attrs):
        el = ET.Element(tag, attrib=attrs)
        if isinstance(cs, list): el.extend([mk_el(*o) for o in cs])
        elif cs is not None: el.text = str(cs)
        return el

    root = mk_el(*node)
    ET.indent(root)
    res = ET.tostring(root, encoding='unicode')
    return hl_md(res) if hl else res

In [ ]:
#| export
def xt(tag, c=None, **kw):
    "Helper to create appropriate data structure for `to_xml`"
    kw = {k.lstrip('_'):str(v) for k,v in kw.items()}
    return tag,c,kw

In [ ]:
#| export
g = globals()
tags = 'div','img','h1','h2','h3','h4','h5','p','hr','span','html'
for o in tags: g[o] = partial(xt, o)

In [ ]:
a = html([
    p('This is a paragraph'),
    hr(),
    xt('x-custom', foo='bar'),
    img(src='http://example.prg'),
    div([
        h1('This is a header'),
        h2('This is a sub-header', style='k:v'),
    ], _class='foo')
])

In [ ]:
to_xml(a, True)

```xml
<html>
  <p>This is a paragraph</p>
  <hr />
  <x-custom foo="bar" />
  <img src="http://example.prg" />
  <div class="foo">
    <h1>This is a header</h1>
    <h2 style="k:v">This is a sub-header</h2>
  </div>
</html>
```

In [ ]:
#|export
def json_to_xml(d:dict, rnm:str)->str:
    "Convert `d` to XML with root name `rnm`"
    root = ET.Element(rnm)
    def build_xml(data, parent):
        if isinstance(data, dict):
            for key, value in data.items(): build_xml(value, ET.SubElement(parent, key))
        elif isinstance(data, list):
            for item in data: build_xml(item, ET.SubElement(parent, 'item'))
        else: parent.text = str(data)
    build_xml(d, root)
    ET.indent(root)
    return ET.tostring(root, encoding='unicode')

In [ ]:
a = dict(surname='Howard', firstnames=['Jeremy','Peter'],
         address=dict(state='Queensland',country='Australia'))
print(json_to_xml(a, 'person'))

<person>
  <surname>Howard</surname>
  <firstnames>
    <item>Jeremy</item>
    <item>Peter</item>
  </firstnames>
  <address>
    <state>Queensland</state>
    <country>Australia</country>
  </address>
</person>


## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()